In [1]:
import pandas as pd
import numpy as np
import sys
import logging
from sklearn.metrics.pairwise import cosine_similarity
import emoji 

sys.path.append("src")
from song_search import embed, find_nearest_song_annoy, lyrics_pre_process
from text_to_emoji import translate_text

In [2]:
print([ascii(i) for i in list('🧱🤔🏗️🧱🔨')])
print(list('🧱🤔🏗️🧱🔨'))
print(ascii('916,🏗️, Building Construction'))

["'\\U0001f9f1'", "'\\U0001f914'", "'\\U0001f3d7'", "'\\ufe0f'", "'\\U0001f9f1'", "'\\U0001f528'"]
['🧱', '🤔', '🏗', '️', '🧱', '🔨']
'916,\U0001f3d7\ufe0f, Building Construction'


In [59]:
lyrics = pd.read_csv('../data/sample_data/top_10_artists_songs.csv')
emojis = pd.read_csv('../data/emoji.csv')

data_text = [
    'Remember those walls I built',
    'Driver roll up the partition please',
    'If I were a Boy',
    'Bring the beat in!',
    '[Intro - Jay Z]',
    'To the left..',
    'What happened at the New Orleans?',
    'Miss Third Wards, your first question',
    'Girls, we run this motha!',
    'Listen,'
]

data_1 = [
    '🧱🤔🏗️🧱🔨',
    '🚗🔒🔝🪜🙏',
    '🤔👦🛡️👧',
    '🎵🥁📢',
    '🎵🎤👑',
    '⬅️',
    '🤔🏙️🤷‍♂️🏙️',
    '👩‍🏫🏫👈❓',
    '👯‍♀️🏃‍♀️',
    '👂🤫'
]

def get_emoji_name(emoji_char):
    emoji_info = emoji.demojize(emoji_char)
    emoji_name = emoji_info.strip(':').replace('\u200d', '')
    if len(emoji_name) > 0:
        return emoji_name
    return ''

# get a song name by the 1st line of lyrics
def measure_accuracy(data, n=10):
    matched = 0
    for i in range(n):
        match = lyrics['song_name'].iloc[i]

        text = lyrics_pre_process(data[i])
        guess = find_nearest_song_annoy(text, 100)
        
        if match in guess:
            matched += 1
    accuracy = matched * 100 / n
    return accuracy

# get a song name by an emoji sequence
def avg_pos(data, n=10):
    matched = []
    for i in range(n):
        match = lyrics['song_name'].iloc[i]

        input_emoji = data[i]

        input_emoji_names = [get_emoji_name(emoji_char) for emoji_char in input_emoji \
            if len(get_emoji_name(emoji_char)) > 0]
        concatenated_names = '.'.join(input_emoji_names)

        # input_emoji_names = [emojis.loc[emojis['emoji'].map(lambda x: emoji in x), 'name'].iloc[0] \
        #     if len(emojis.loc[emojis['emoji'].map(lambda x: emoji in x), 'name']) else 'Unknown' for emoji in input_emoji]
        # concatenated_names = input_emoji

        guess = find_nearest_song_annoy(concatenated_names, 10000) + [match]
        print(guess.index(match), concatenated_names)
        matched.append(guess.index(match))

    return np.mean(matched)

def test_accuracy_lyrics2emoji_search():
        accuracy = avg_pos(data_1)
        logging.critical(f'mean searched pos: {accuracy}')



In [60]:
test_accuracy_lyrics2emoji_search()

21 brick.thinking_face.building_construction.brick.hammer
74 automobile.locked.TOP_arrow.ladder.folded_hands
193 thinking_face.boy.shield.girl
86 musical_note.drum.loudspeaker
729 musical_note.microphone.crown
5 left_arrow
513 thinking_face.cityscape.person_shrugging.male_sign.cityscape


CRITICAL:root:mean searched pos: 234.7


0 woman.school.school.backhand_index_pointing_left.red_question_mark
641 people_with_bunny_ears.female_sign.person_running.female_sign
85 ear.shushing_face
